In [92]:
import requests
import re
from bs4 import BeautifulSoup

In [12]:
def get_urls_by_type():
    '''Scrapes arkhamdb and returns a list of urls pages containing card information
       on each card for each card type'''

    # call function to return list of url's by catagory
    print("Getting investigator URL's...")
    gators = get_cata_url('investigator')

    print("Getting asset URL's...")
    assets = get_cata_url('asset')

    print("Getting event URL's...")
    events = get_cata_url('event')

    print("Getting skill URL's...")
    skills = get_cata_url('skill')

    return gators, assets, events, skills

def get_cata_url(catagory):
    ''' Takes in a string containing a player card faction, Scrapes arkhamdb, and
        Returns a list of urls for containing information on each card of that catagory'''

    full_results = []

    # defines number of results pages to scrape
    if catagory in ('investigator', 'skill'):
        
        pages = 1
        
    elif catagory == 'event':
        
        pages = 2
        
    elif catagory == 'asset':
        
        pages = 4

    # itterates through pages in catagory
    for page in range(1,pages+1):

        url = f'https://arkhamdb.com/find?q=t%3A{catagory}&view=list&sort=name&decks=player&page={page}'

        # create request and soup objects
        html = requests.get(url)

        soup = BeautifulSoup(html.content, 'html.parser')

        # locate urls on page
        results = soup.find(id='list')

        results = results.find_all('a', class_='card-tip')

        # convert urls to string and make a list
        results = [str(result['href']) for result in results]
                
        full_results.extend(results)
                
    return full_results

def get_soup(url):
    '''Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page and return
    return soup

In [61]:
def get_title(soup): 
    
    title = soup.find('a', class_='card-name card-tip').text.replace('\n', '')
    
    try:
        
        subtitle = soup.find('div', class_='card-subname small').text
        
    except:
        
        subtitle = ''
    
    
    return title + ' ' + subtitle


def get_faction(soup):
    
    faction = ''

    for item in soup.find_all('div', class_='card-faction'):

        faction += item.text.replace('\n', ' ')

    return faction

In [129]:
def get_text_for_icon(text):
    '''Takes in request response as a string
       replaces html code indicating a game icon with a text representation
       Returns sting with replacements'''
    
    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text

In [4]:
gators, assets, events, skills = get_urls_by_type()

Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [118]:
gator = get_soup(gators[3])
asset = get_soup('https://arkhamdb.com/card/08109')
event = get_soup(events[15])
skill = get_soup(skills[56])

soups = [gator,asset,event,skill]

In [71]:
for soup in soups:
    
    print(soup.find('p', class_='card-traits').text)

Sorcerer.
Item. Weapon. Firearm. Illicit.
Spirit.
Practiced.


In [44]:
title = get_title(soup)

faction = get_faction(soup)

tipe = soup.find('span', class_='card-type').text

traits = soup.find('p', class_='card-traits').text

willpower = soup.find('li', title='Willpower').text

intellect = soup.find('li', title='Intellect').text

combat = soup.find('li', title='Combat').text

agility = soup.find('li', title='Agility').text

health = re.search('Health:\s+(\d)', str(soup.find('div'))).group(1)

sanity = re.search('Sanity:\s+(\d)', str(soup.find('div'))).group(1)

ability = 

TypeError: find() takes no keyword arguments

In [125]:
faction = get_faction(asset).replace('  ',' ').lower().strip()

faction

['rogue', 'mystic']

In [130]:
faction = get_faction(asset).strip().lower()

first_faction = faction.split(' ')[0]

soup = asset.find('div', class_=f'card-text border-{first_faction}')

ability = get_text_for_icon(soup)

ability

TypeError: 'NoneType' object is not callable

In [100]:
re.search('Sanity:\s+(\d)', str(gator.find('div'))).group(1

'8'

In [38]:
asset.find_all('span', class_='card-faction')

[<span class="card-faction">Rogue</span>]